# Answer to graded assignment 2 in DTE-2501 (AI Methods and Applications) about ensemble methods by Abdullah Karagøz

## 1. Bootstrapping

In [1]:
import pandas as pd
import numpy as np
from math import sqrt
from math import pi
from math import exp
from math import log
#import platform
#platform.architecture()

seed = 15

# Load the data
dataset = pd.read_csv('iris.data', header=None, names=['sepal length', 'sepal width',
                                                     'petal length', 'petal width',
                                                     'class'])

In [2]:
#Assign label to each class
dataset.loc[dataset['class'] == 'Iris-setosa', dataset.columns == 'class'] = 0
dataset.loc[dataset['class'] == 'Iris-versicolor', dataset.columns == 'class'] = 1
dataset.loc[dataset['class'] == 'Iris-virginica', dataset.columns == 'class'] = 2

In [3]:
train_test_split = 0.8

# Shuffle and split data into training 80% and testing 20%
train_set = dataset.sample(frac=train_test_split, random_state=seed)
test_set = dataset.drop(train_set.index)
train_set.reset_index(drop=True, inplace=True)
test_set.reset_index(drop=True, inplace=True)



In [4]:
train_set[train_set['class']==0].iloc[:,0].values.tolist()

train_set

,sepal length,sepal width,petal length,petal width,class
0,4.6,3.4,1.4,0.3,0
1,5.9,3.0,4.2,1.5,1
2,5.5,2.6,4.4,1.2,1
3,4.8,3.1,1.6,0.2,0
4,5.4,3.4,1.5,0.4,0
...,...,...,...,...,...
115,4.6,3.6,1.0,0.2,0
116,5.0,3.5,1.3,0.3,0
117,7.2,3.2,6.0,1.8,2
118,4.5,2.3,1.3,0.3,0


### Naive Bayes Classifier

In [5]:
# Naive Bayes Classifier
class NaiveBayesClassifier():
    def __init__(self):
        self.dataset = 0
        self.mean_values = 0
        self.std_values = 0
        self.prior_class_probabilities = 0
        self.nr_of_classes = 0
        self.nr_of_attributes = 0
        
    def fit(self, train_set, in_ensemble=False, nr_of_classes = 0,
            nr_of_attributes=0):
        self.dataset = train_set
        
        # calculating mean, std values and prior probabilities
        
        # if not in ensemble, we calculate these values
        if not in_ensemble:
            self.nr_of_classes=train_set.groupby('class').ngroups
            self.nr_of_attributes=len(train_set.iloc[0,:-1])
        else:
            self.nr_of_classes = nr_of_classes
            self.nr_of_attributes = nr_of_attributes
        
        # Here we keep mean, std and prior class probability values
        self.mean_values = list()
        self.std_values = list()
        self.prior_class_probs = list()
        
        for i in range(self.nr_of_classes):
            class_values = train_set[train_set['class'] == i]
            prior_class_prob = len(class_values) / len(train_set)
            self.prior_class_probs.append(prior_class_prob)
            mean_values = list()
            std_values = list()   
            for j in range(self.nr_of_attributes):
                values = class_values.iloc[:,j].values.tolist()
                std, mean = self.std_and_mean(values)
                mean_values.append(mean)
                std_values.append(std)
            self.mean_values.append(mean_values)
            self.std_values.append(std_values)


    def mean(self, val_list):
        return sum(val_list) / len(val_list)
    
    def std_and_mean(self, val_list):
        mu = self.mean(val_list)
        std = sqrt(sum([(x - mu)**2 for x in val_list]) / (len(val_list)-1))
        return std, mu
        
    
    def gaussian_pdf(self, x, mu, sd):
        return_val = (1 / (sd*sqrt(2*pi))) * exp(-0.5*((x-mu)/sd)**2)
        return return_val
    
    
    def pred_row(self, row, log_prob = False):
        probs = self.nr_of_classes*[0]  
        if log_prob:
            for i in range(self.nr_of_classes):
                probs[i] = 0 if self.prior_class_probs[i] == 0 else log(self.prior_class_probs[i])
                for j in range(len(row)):
                    gaussian_val = self.gaussian_pdf(row[j], self.mean_values[i][j], self.std_values[i][j])
                    probs[i] += -float('inf') if gaussian_val == 0 else log(gaussian_val)
        
        else: 
            for i in range(self.nr_of_classes):
                probs[i] = self.prior_class_probs[i]
                for j in range(len(row)):
                    probs[i] *= self.gaussian_pdf(row[j], self.mean_values[i][j], self.std_values[i][j])
        
        prediction = probs.index(max(probs))
        return probs, prediction   
    
    def predict(self, test_set, log_prob = False):
        X_test = test_set.iloc[:, :-1].values.tolist()
        Y_test = test_set.iloc[:, -1].values.tolist()
        probabilities = list()
        predictions = list()
        corrects = 0
        for x, y in zip(X_test, Y_test):
            prob, pred = self.pred_row(x, log_prob)
            probabilities.append(prob)
            predictions.append(pred)
            if pred == y:
                corrects += 1    
        accuracy = corrects / len(Y_test)
        return probabilities, accuracy, predictions

### Ensembled Classifier

In [6]:
class EnsembledNBClassifier():
    def __init__(self, nr_of_classes):
        self.classifiers = list()
        for i in range(nr_of_classes):
            self.classifiers.append(NaiveBayesClassifier())
    
    def fit(self, train_set, seed=None):
        self.nr_of_classes=train_set.groupby('class').ngroups
        self.nr_of_attributes=len(train_set.iloc[0,:-1])
        for cl in self.classifiers:
            bag = train_set.sample(frac=1, replace=True, random_state=seed).reset_index(drop=True)
            cl.fit(bag, True, self.nr_of_classes, self.nr_of_attributes)
        
    
    def predict(self, test_set, log_prob = False, majority_vote = False):
        X_test = test_set.iloc[:, :-1].values.tolist()
        Y_test = test_set.iloc[:, -1].values.tolist()

        corrects = 0
        for x, y in zip(X_test, Y_test):
            preds = list()
            probs = list()

            for cl in self.classifiers:
                prob, pred = cl.pred_row(x, log_prob)
                preds.append(pred)
                probs.append(prob)
            if majority_vote:
                prediction = max(set(preds), key = preds.count)
            else:
                if log_prob:           
                    agg_probs = [sum(x)/len(probs) for x in zip(*probs)]
                else:
                    agg_probs = self.nr_of_classes*[1]
                    for i in range(len(probs[0])):
                        for j in range(len(probs)):
                            agg_probs[i] *= probs[j][i]
                        agg_probs[i] = agg_probs[i]**(1/len(probs))
                prediction = agg_probs.index(max(agg_probs))
            if prediction == y:
                corrects += 1
                
                
        accuracy = corrects / len(Y_test)
        return accuracy 
        
        

## Testing and validation

In [46]:
train_test_split = 0.4

# Shuffle and split data into training 80% and testing 20%
train_set = dataset.sample(frac=train_test_split, random_state=None)
test_set = dataset.drop(train_set.index)
train_set.reset_index(drop=True, inplace=True)
test_set.reset_index(drop=True, inplace=True)

nbc = NaiveBayesClassifier()
nbc.fit(train_set)

probabilities, accuracy, preds = nbc.predict(test_set, log_prob=True)
preds = np.array(preds)

probabilities2, accuracy2, preds2 = nbc.predict(test_set, log_prob=False)
preds2 = np.array(preds)

from sklearn.naive_bayes import GaussianNB

X_train = train_set.iloc[:,:-1].to_numpy()
Y_train = train_set.iloc[:,-1].to_numpy().astype('int')
X_test = test_set.iloc[:,:-1].to_numpy()
Y_test = test_set.iloc[:,-1].to_numpy().astype('int')

gnb = GaussianNB()
gnb.fit(X_train, Y_train)

Y_pred = gnb.predict(X_test)

print("Accuracy with sklearn classifier", (Y_test == Y_pred).sum() / len(Y_test))
print("Accuracy with my classifier with log prob", accuracy)
print("Accuracy with my classifier without log prob", accuracy2)
print("Predictions with sklearn classifier", Y_pred)
print("Predictions with log prob", preds)
print("Predictions without log prob", preds2)
print("Values from test set", Y_test)

ens_nbc = EnsembledNBClassifier(10)
ens_nbc.fit(train_set)

accuracy = ens_nbc.predict(test_set)
print("Ensemble accuracy with aggregation", accuracy)

accuracy = ens_nbc.predict(test_set, log_prob=True)
print("Ensemble accuracy with log and aggregation", accuracy)

accuracy = ens_nbc.predict(test_set, majority_vote=True)
print("Accuracy with majority vote", accuracy)

accuracy = ens_nbc.predict(test_set, log_prob=True, majority_vote=True)
print("Accuracy with log and majority vote", accuracy)

Accuracy with sklearn classifier 0.9666666666666667
Accuracy with my classifier with log prob 0.9666666666666667
Accuracy with my classifier without log prob 0.9666666666666667
Predictions with sklearn classifier [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2
 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Predictions with log prob [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2
 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Predictions without log prob [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2
 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Values from test set [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 

In [9]:
arr = [1,2,3,4]
arr[2] = arr[2]**2
arr

[1, 2, 9, 4]

int